# BILBY example with BNS data

In [12]:
import numpy as np
import bilby, h5py
from gwpy.timeseries import TimeSeries

In [1]:
bns_filename = '/ceph/submit/data/user/k/kyoon/KYoonStudy/models/BNS/bns_waveforms.hdf5'

In [ ]:
# Set the interferometers
H1 = bilby.gw.detector.get_empty_interferometer('H1')
L1 = bilby.gw.detector.get_empty_interferometer('L1')

In [13]:
# Fetch the timeseries strain data
with h5py.File(bns_filename, 'r') as f:
    # === Access metadata ===
    coalescence_time = f.attrs['coalescence_time'] # GPS time of coalescence
    duration = f.attrs['duration'] # Duration of the waveform in seconds
    ifos = f.attrs['ifos'] # List of interferometers
    length = f.attrs['length'] # Number of samples.
    num_injections = f.attrs['num_injections'] # Number of waveform injections.
    sample_rate = f.attrs['sample_rate'] # Sample rate in Hz

    # === Choose one injection (e.g., the 0th) ===
    waveform_idx = 0
    h1_strain = f['waveforms/h1'][waveform_idx]
    l1_strain = f['waveforms/l1'][waveform_idx]
    m1 = f['parameters/mass_1'][waveform_idx]
    m2 = f['parameters/mass_2'][waveform_idx]
    chirp_mass = (m1+m2)**(0.6)/(m1+m2)**(0.2)

    start_time = coalescence_time - duration
    times = np.linspace(start_time, coalescence_time, int(duration * sample_rate))

    h1_ts = TimeSeries(h1_strain, times=times, name='H1')
    l1_ts = TimeSeries(l1_strain, times=times, name='L1')

In [27]:
# Set strain data directly for each interferometer
H1.set_strain_data_from_gwpy_timeseries(h1_ts)
L1.set_strain_data_from_gwpy_timeseries(l1_ts)

In [29]:
# Load priors
prior = bilby.gw.prior.BNSPriorDict()

16:13 bilby INFO    : No prior given, using default BNS priors in /work/submit/kyoon/miniforge3/envs/ssm/lib/python3.10/site-packages/bilby/gw/prior_files/aligned_spins_bns_tides_on.prior.


In [30]:
prior

{'mass_1': Constraint(minimum=0.5, maximum=5, name='mass_1', latex_label='$m_1$', unit=None),
 'mass_2': Constraint(minimum=0.5, maximum=5, name='mass_2', latex_label='$m_2$', unit=None),
 'mass_ratio': bilby.gw.prior.UniformInComponentsMassRatio(minimum=0.125, maximum=1, name='mass_ratio', latex_label='$q$', unit=None, boundary=None, equal_mass=False),
 'chirp_mass': bilby.gw.prior.UniformInComponentsChirpMass(minimum=0.4, maximum=4.4, name='chirp_mass', latex_label='$\\mathcal{M}$', unit=None, boundary=None),
 'luminosity_distance': bilby.gw.prior.UniformSourceFrame(minimum=100.0, maximum=5000.0, cosmology='Planck15', name='luminosity_distance', latex_label='$d_L$', unit='Mpc', boundary=None),
 'dec': Cosine(minimum=-1.5707963267948966, maximum=1.5707963267948966, name='dec', latex_label='$\\mathrm{DEC}$', unit=None, boundary=None),
 'ra': Uniform(minimum=0, maximum=6.283185307179586, name='ra', latex_label='$\\mathrm{RA}$', unit=None, boundary='periodic'),
 'theta_jn': Sine(minimum=

In [ ]:
### TODO: fix ###
# Set up the likelihood (example: using a standard GW likelihood)
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=bilby.gw.detector.networks.InterferometerList([H1, L1]),
    waveform_generator=None,  # You need to set up a waveform generator as per your data/model
    time_marginalization=True,
    phase_marginalization=True,
    distance_marginalization=True,
)

# Run the sampler to infer posteriors
result = bilby.run_sampler(
    likelihood=likelihood,
    priors=prior,
    sampler='dynesty',
    nlive=500,
    outdir='outdir',
    label='bns_example'
)